In [1]:
# Import GYM stuff
import gym
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete

# Import helpers
import numpy as np
import random
import os

# Import Stable baselines stuff
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

In [2]:
Discrete(3).sample()

0

In [3]:
Box(0,1, shape=(3,)).sample()

array([0.6868575 , 0.9701208 , 0.64844745], dtype=float32)

In [4]:
Tuple((Discrete(3), Box(0,1, shape=(3,)))).sample()

(0, array([0.4337252 , 0.11864923, 0.9870596 ], dtype=float32))

In [5]:
Dict({'height':Discrete(2), 'speed':Box(0,100,shape=(1,))}).sample()

OrderedDict([('height', 0), ('speed', array([58.794296], dtype=float32))])

In [6]:
MultiDiscrete([5,2,2,5]).sample()

array([2, 1, 0, 1])

# Building an Environment
 - Build an egent to give us the best shower possible
 - Randomly temperature
 - 37 and 39 degrees

In [7]:
class ShowerEnv(Env):
    def __init__(self):
        self.action_space = Discrete(3)
        self.observation_space = Box(low=0, high=100, shape=(1,))
        self.state = 38 + random.randint(-3, 3)
        self.shower_length = 60
    def step(self, action):
        # Apply temp adj
        self.state += action-1
        
        #Decrease shower time
        self.shower_length -= 1
        
        # Calculate Reward
        if(self.state >= 37 and self.state <= 39):
            reward = 1
        else:
            reward = -1
            
        if(self.shower_length <= 0):
            done = True
        else:
            done = False
        
        info = {}
        
        return self.state, reward, done, info

    def render(self):
        pass

    def reset(self):
        self.state = np.array([38+random.randint(-3, 3)]).astype(float)
        self.shower_length = 60
        return self.state

# Test Environment

In [8]:
env = ShowerEnv()

In [9]:
env.observation_space

Box([0.], [100.], (1,), float32)

In [10]:
env.action_space

Discrete(3)

In [11]:
episodes = 5
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0
    while not done:
        env.render()
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:22
Episode:2 Score:4
Episode:3 Score:10
Episode:4 Score:-60
Episode:5 Score:-46


# Train Model

In [12]:
log_path = os.path.join('/home/kchn/rlp/custom_env/Training', 'Logs')
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [15]:
model.learn(total_timesteps=20000)

Logging to /home/kchn/rlp/custom_env/Training/Logs/PPO_3
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | 1.41     |
| time/              |          |
|    fps             | 1905     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | 8.15        |
| time/                   |             |
|    fps                  | 1296        |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.007958722 |
|    clip_fraction        | 0.152       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.979      |
|    explained_

# Save model

In [16]:
shower_path = os.path.join('/home/kchn/rlp/custom_env/Training', 'Saved Models', 'Shower_Model_PPO')

In [17]:
model.save(shower_path)

/home/kchn/.local/lib/python3.10/site-packages/stable_baselines3/common/save_util.py:276: UserWarning: Path '/home/kchn/rlp/custom_env/Training/Saved Models' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


In [25]:
del model

In [26]:
model = PPO.load(shower_path, env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [27]:
evaluate_policy(model, env, n_eval_episodes=10)

(23.6, 54.73426714591143)